<a href="https://colab.research.google.com/github/jeongwoo22/ESAA_assignments/blob/main/OB_%EC%84%B8%EC%85%981_0905.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 세션1_0905
### 핸즈온 머신러닝 3.8 연습문제 풀기

### **1. MNIST 데이터셋으로 분류기를 만들어 테스트 세트에서 97% 정확도를 달성해보세요.**
___

1. `KNeghtborsClassifier`를 사용하는 것을 추천합니다. 
2. `weights`와 `n_neighbors` 하이퍼 파라미터로 그리드 탐색을 시도하여, 좋은 하이퍼 파라미터 값을 찾아보세요.

In [1]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1)
mnist.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [2]:
X,y = mnist['data'], mnist['target']

In [3]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

In [6]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

knn_clf = KNeighborsClassifier()
parameter = [{'weights':["uniform", "distance"], 'n_neighbors':[3,4,5]}]

grid_search = GridSearchCV(knn_clf, parameter, cv=5, verbose=3)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END ....n_neighbors=3, weights=uniform;, score=0.972 total time=  36.6s
[CV 2/5] END ....n_neighbors=3, weights=uniform;, score=0.971 total time=  42.7s
[CV 3/5] END ....n_neighbors=3, weights=uniform;, score=0.969 total time=  41.5s
[CV 4/5] END ....n_neighbors=3, weights=uniform;, score=0.969 total time=  34.2s
[CV 5/5] END ....n_neighbors=3, weights=uniform;, score=0.970 total time=  34.4s
[CV 1/5] END ...n_neighbors=3, weights=distance;, score=0.972 total time=  34.2s
[CV 2/5] END ...n_neighbors=3, weights=distance;, score=0.972 total time=  34.6s
[CV 3/5] END ...n_neighbors=3, weights=distance;, score=0.970 total time=  34.1s
[CV 4/5] END ...n_neighbors=3, weights=distance;, score=0.970 total time=  33.8s
[CV 5/5] END ...n_neighbors=3, weights=distance;, score=0.971 total time=  33.9s
[CV 1/5] END ....n_neighbors=4, weights=uniform;, score=0.969 total time=  37.7s
[CV 2/5] END ....n_neighbors=4, weights=uniform;,

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid=[{'n_neighbors': [3, 4, 5],
                          'weights': ['uniform', 'distance']}],
             verbose=3)

In [8]:
y_pred = grid_search.predict(X_test)

In [9]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9714

### **2. 다음 단계를 따라 인위적으로 훈련 세트를 늘리는 데이터 증식 또는 훈련 세트 확장 기법을 연습해봅시다.**
___

In [10]:
from scipy.ndimage.interpolation import shift

In [11]:
def shift_image(image, dx, dy):
    image = image.reshape((28, 28))
    shifted_image = shift(image, [dy, dx], cval=0, mode="constant")
    return shifted_image.reshape([-1])

In [ ]:
image = X_train[1000]
shifted_image_down = shift_image(image, 0, 5)
shifted_image_left = shift_image(image, -5, 0)

In [13]:
plt.figure(figsize=(12,3))
plt.subplot(131)
plt.title("Original", fontsize=14)
plt.imshow(image.reshape(28, 28), interpolation="nearest", cmap="Greys")
plt.subplot(132)
plt.title("Shifted down", fontsize=14)
plt.imshow(shifted_image_down.reshape(28, 28), interpolation="nearest", cmap="Greys")
plt.subplot(133)
plt.title("Shifted left", fontsize=14)
plt.imshow(shifted_image_left.reshape(28, 28), interpolation="nearest", cmap="Greys")
plt.show()

In [ ]:
X_train_augmented = [image for image in X_train]
y_train_augmented = [label for label in y_train]

for dx, dy in ((1, 0), (-1, 0), (0, 1), (0, -1)):
    for image, label in zip(X_train, y_train):
        X_train_augmented.append(shift_image(image, dx, dy))
        y_train_augmented.append(label)

X_train_augmented = np.array(X_train_augmented)
y_train_augmented = np.array(y_train_augmented)

In [ ]:
shuffle_idx = np.random.permutation(len(X_train_augmented))
X_train_augmented = X_train_augmented[shuffle_idx]
y_train_augmented = y_train_augmented[shuffle_idx]

In [ ]:
knn_clf = KNeighborsClassifier(**grid_search.best_params_)

In [ ]:
knn_clf.fit(X_train_augmented, y_train_augmented)

In [ ]:
y_pred = knn_clf.predict(X_test)
accuracy_score(y_test, y_pred)